# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [26]:
#Question 1

def square_root_generator(limit):
    n = 1
    while n <= limit: #after yielding the result, it'll increment n by 1 until it reaches 5
        yield n ** 0.5 
        n += 1


limit = 5 # max value n can take
generator = square_root_generator(limit)

#to answer q 1 and get sum of outputs
count = []

for sqrt_value in generator:
    count.append(sqrt_value)


#sum
print(sum(num for num in count))


8.382332347441762


In [27]:
#Question 2

def square_root_generator(limit):
    n = 1
    while n <= limit: 
        yield n ** 0.5 
        n += 1


limit = 13 # max value n can take
generator = square_root_generator(limit)

count = []

for sqrt_value in generator:
    count.append(sqrt_value)

#13 num. yielded
print(count[12])

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [63]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [54]:
import dlt
import duckdb

people_1_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_1_gen')

#load data
people_1_pipeline.run(people_1(),
                    table_name='people_1_table',
                    write_disposition="replace")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x00000226606A1150>, metrics={'1707651569.663776': [{'started_at': DateTime(2024, 2, 11, 11, 39, 30, 45570, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 11, 11, 39, 30, 278213, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:///C:\\Users\\44751\\Documents\\GitHub\\DTC-Data-Engineering-2024\\Workshops\\dlt - data ingestion\\dlt_resources\\dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people_1_gen', loads_ids=['1707651569.663776'], load_packages=[LoadPackageInfo(load_id='1707651569.663776', package_path='C:\\Users\\44751\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1707651569.663776', state='loaded', schema=Schema dlt_ipykernel_launcher at 2363850536800, schema_update={}, completed_at=DateTim

In [60]:

conn = duckdb.connect(f"{people_1_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{people_1_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

#Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
question_2 = conn.sql("SELECT sum(age) AS sum_age FROM people_1_table")
display(question_2)




Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1_table      │
└─────────────────────┘

┌─────────┐
│ sum_age │
│ int128  │
├─────────┤
│     140 │
└─────────┘

In [61]:
#Append the second generator to the same table as the first.
people_1_pipeline.run(people_2(),
                    table_name='people_1_table',
                    write_disposition="append")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x00000226606A1150>, metrics={'1707652579.068459': [{'started_at': DateTime(2024, 2, 11, 11, 56, 19, 467716, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 11, 11, 56, 19, 719974, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:///C:\\Users\\44751\\Documents\\GitHub\\DTC-Data-Engineering-2024\\Workshops\\dlt - data ingestion\\dlt_resources\\dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people_1_gen', loads_ids=['1707652579.068459'], load_packages=[LoadPackageInfo(load_id='1707652579.068459', package_path='C:\\Users\\44751\\.dlt\\pipelines\\dlt_ipykernel_launcher\\load\\loaded\\1707652579.068459', state='loaded', schema=Schema dlt_ipykernel_launcher at 2363832835504, schema_update={'people_1_table': {'na

In [62]:
#Question 3
question_3 = conn.sql("SELECT sum(age) AS sum_age FROM people_1_table")
display(question_3)

┌─────────┐
│ sum_age │
│ int128  │
├─────────┤
│     353 │
└─────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [65]:
#load table 1 again
people_1_pipeline.run(people_1(),
                    table_name='people_1_table',
                    write_disposition="replace")

#load table 2 - with merge
people_1_pipeline.run(people_2(),
                    table_name='people_1_table',
                    write_disposition="merge",
                    primary_key="ID")

question_4 = conn.sql("SELECT sum(age) AS sum_age FROM people_1_table")
display(question_4)

┌─────────┐
│ sum_age │
│ int128  │
├─────────┤
│     266 │
└─────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX